In [ ]:
import numpy as np

def print_m(M):
    print('[', ',\n  '.join(map(str, M)) , '],',)


def generate_rotrefl(tile):
    out = []
    tile = np.array(tile)
    for j in range(6):
        tile = np.rot90(tile)
        if tile.tolist() not in out:out.append(tile.tolist())
        if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
        if np.transpose(tile).tolist() not in out:out.append(np.transpose(tile).tolist())
    return out



def draw_from_tset(p):
    s = []
    for i in range(len(p)):
        t = tiles_sets[p[i][0]][0]
        for a,b in t:
            s.append((a + p[i][1],b + p[i][2], i+1))
    minx, maxx = min(s, key = lambda t:t[0])[0], max(s, key = lambda t:t[0])[0]
    miny, maxy = min(s, key = lambda t:t[1])[1], max(s, key = lambda t:t[1])[1]
    t = np.zeros((maxx-minx + 1 , maxy-miny + 1), dtype = int)
    for i in s: t[i[0]-minx][i[1]-miny] = i[2]
    return t.tolist()


def get_coord_from_m(m):
    out = set()
    orig = (0,0)
    c = 0
    for i in range(len(m)):
        if c == 1:break
        for j in range(len(m[i])):
            if m[i][j] != 0:
                c = 1
                orig = (i,j)
                break
                
    for i in range(len(m)):
        for j in range(len(m[i])):
            if  m[i][j] != 0:
                out.add((i-orig[0],j-orig[0]))
    return out


def get_neighbourhood(s):
    out = s.copy()
    def near(x,y):
        return set(((x,y+1),(x+1,y),(x,y-1),(x-1,y)))
    for i in s:
        out = out | near(*i)
    return out - s 


def extract_m_from_set_directly(s):
    minx = min(s, key = lambda t:t[0])[0]
    maxx = max(s, key = lambda t:t[0])[0]
    miny = min(s, key = lambda t:t[1])[1]
    maxy = max(s, key = lambda t:t[1])[1]
    t =  [[0]*(maxy-miny+1) for  i in  range(maxx-minx+1)]
    for i in s:t[i[0]-minx][i[1]-miny] = 1
    return t


def gen_batch(tiles):
    out = []
    for i in tiles:
        for j in generate_rotrefl(i):
            dt = get_coord_from_m(j)
            out.append((dt, get_neighbourhood(dt)))
    return out
    

def connected_tile(k):
    s = k.copy()
    def near(x,y):
        return ((x,y+1),(x+1,y),(x-1,y),(x,y-1),)
    def start_from_seed(x,y):
        if (x,y) not in s:
            return
        s.discard((x,y))
        for dx, dy in near(x, y):
            start_from_seed(dx, dy)
    start_from_seed(*tuple(s)[0])
    return len(s) == 0


def get_border(K):
    from copy import deepcopy
    M = deepcopy(K)
    d1  =  len(M)
    d2  =  len(M[0])
    def near(x,y): return ((x,y+1),(x+1,y),(x-1,y),(x,y-1),)
    def start_from_seed(x,y,M):
        if not(0 <= x < d1 and 0 <= y < d2) or M[x][y] == 2 or M[x][y] == 3: return
        if M[x][y] == 1:
            M[x][y] = 2
            return
        M[x][y] = 3
        for dx, dy in near(x, y):
            start_from_seed(dx, dy, M)
    for i in range(d1):
        start_from_seed(i, 0, M)
        start_from_seed(i, d2-1, M)
    for i in range(d2):
        start_from_seed(0,i, M)
        start_from_seed(d1-1, i, M)
    for x in range(len(M)):
        for y in range(len(M[0])):
            if M[x][y] == 2: 
                M[x][y] = 1
            else: 
                M[x][y] = 0
    return M



In [ ]:


T1 = [[1,0,0],
      [1,1,0],
      [0,0,0],]

T2 = [[1,0],
      [1,1],]

tiles = [T1,]


tiles = [j for i in tiles for j in generate_rotrefl(i)]
tiles_sets =[(get_coord_from_m(i), get_neighbourhood(get_coord_from_m(i))) for i in tiles]


def tile_rect(n,m, tiles):
    global border
    maxntest = 20000

    grid = set([(i,j) for i in range(n) for j in range(m)])
    border = set()

    for i in range(n):
        border.add((i, 0))
        # border.add((i, m-1))

    for i in range(m):
        border.add((0,i))
        # border.add((n-1, i))

    global out
    global end
    global memo

    end = []
    out = []
    memo = set()

    def dfs(grid, border, pattern, neighbourhood, p, n):
        
        
        if len(end) > 0:
            return

        if pattern != set():
            
            h = extract_m_from_set_directly(pattern)

            print(n)
            print_m(h)
            print('\n border ')
            print_m(extract_m_from_set_directly(border))

            # if pattern != set() and has_a_hole(h):
            #     return

            if grid != set() and not connected_tile(grid):
                return

            if grid == set():
                out.append(p)
                end.append(1)
                return

            # sh = int(''.join(''.join(map(str, i)) for i in h) )

            # if sh in memo:
            #     return
            # else:
            #     memo.add(sh)

        if grid & border != set():

            for x,y in neighbourhood & border:

                for r in range(len(tiles)):
                    t = tiles[r]
                    tempt = set((x+t1,y+t2) for t1, t2 in t[0])
                    if len(tempt-grid) == 0:
                        temp_p = p.copy()
                        tempnear = set((x+t1,y+t2) for t1, t2 in t[1])
                        temp_p.append((r, x, y))
                        temp_pattern = pattern |  tempt
                        tempgrid = grid - tempt
                        dfs(tempgrid, border,  temp_pattern, ((neighbourhood | tempnear) - temp_pattern) & tempgrid, temp_p, n+1)
        else:

            if grid == set():
                out.append(p)
                end.append(1)
                return

            tempborder = get_coord_from_m(get_border(extract_m_from_set_directly(grid)))

            print_m(get_border(extract_m_from_set_directly(grid)))

            dfs(grid, tempborder, pattern, neighbourhood, p, n)


    startborder =  get_coord_from_m(get_border(extract_m_from_set_directly(grid)))
    dfs(grid, startborder, set(), set(((0,0),)), [], 0) 

    return out



solution = tile_rect(4, 6, tiles_sets)

print(solution)
if solution != []:
    ms = draw_from_tset(solution[0])
    print('[', ',\n  '.join(map(str, ms)) , '],',)


In [ ]:

#broken code

T1 = [[1,0,0],
      [1,1,0],
      [0,0,0],]

T2 = [[1,0],
      [1,1],]

tiles = [T2,]


tiles = [j for i in tiles for j in generate_rotrefl(i)]
tiles_sets =[(get_coord_from_m(i), get_neighbourhood(get_coord_from_m(i))) for i in tiles]


def tile_rect(n,m, tiles):
    global border
    maxntest = 20000

    grid = set([(i,j) for i in range(n) for j in range(m)])
    border = set()

    for i in range(n):
        border.add((i, 0))
        border.add((i, m-1))

    for i in range(m):
        border.add((0,i))
        border.add((n-1, i))

    global out
    global end
    global memo

    end = []
    out = []
    memo = set()


    def border_tiled(b, g):
        return all( i not in g for i in b)



    def dfs(grid, border, pattern, neighbourhood, p, n):
        
        if len(end) > 0:
            return

        if pattern != set():
            
            h = extract_m_from_set_directly(pattern)

            if n == 6: 
                print('----------------------')
                print(n)
                print_m(h)
                print('\n border ')
                print_m(extract_m_from_set_directly(border))
            

            # if pattern != set() and has_a_hole(h):
            #     return

            if len(grid) == 0:
                out.append(p)
                end.append(1)
                return

            if len(grid) > 0 and not connected_tile(grid):
                return

           


            # sh = int(''.join(''.join(map(str, i)) for i in h) )
            # if sh in memo:
            #     return
            # else:
            #     memo.add(sh)



        if len(grid - pattern) > 0:

            for x,y in (neighbourhood & border):

                for r in range(len(tiles)):
                    t = tiles[r]
                    tempt = set((x+t1,y+t2) for t1, t2 in t[0])

                    if set(tempt) <= set(grid):

                        temp_p = p.copy()

                        tempnear = set((x+t1,y+t2) for t1, t2 in t[1])

                        temp_p.append((r, x, y))

                        temp_pattern = pattern |  tempt

                        tempgrid = grid - tempt

                        dfs(tempgrid, border,  temp_pattern, ((neighbourhood | tempnear) - temp_pattern) & tempgrid, temp_p, n+1)
        else:

            if grid == set():
                out.append(p)
                end.append(1)
                return

            tempborder = get_coord_from_m(get_border(extract_m_from_set_directly(grid)))

            print(n)
            print( border  & grid)
            print_m(extract_m_from_set_directly(tempborder))

            dfs(grid, tempborder, pattern, neighbourhood, p, n+1)


    startborder =  get_coord_from_m(get_border(extract_m_from_set_directly(grid)))

    dfs(grid, startborder, set(), set(((0,0),)), [], 0) 

    return out



solution = tile_rect(4, 6, tiles_sets)

print(solution)
if solution != []:
    ms = draw_from_tset(solution[0])
    print('[', ',\n  '.join(map(str, ms)) , '],',)


[]


In [ ]:
from copy import deepcopy

def extract_m_from_set_directly222(s):
    minx = min(s, key = lambda t:t[0])[0]
    maxx = max(s, key = lambda t:t[0])[0]
    miny = min(s, key = lambda t:t[1])[1]
    maxy = max(s, key = lambda t:t[1])[1]

    t =  [[0]*(maxy + 1) for  i in  range(maxx + 1)]

    for i in s:t[i[0]][i[1]] = 1

    return t

def get_coord_from_m_same(m):
    out = set()

    for i in range(len(m)):
        for j in range(len(m[i])):
            if  m[i][j] != 0:
                out.add((i,j))
    return out



def get_coord_from_m_all_displacements(m):
    out = []
    c = 0
    for i in range(len(m)):
        for j in range(len(m[i])):
            if m[i][j] != 0:
                orig = (i,j)

                temp  = set()
                for i in range(len(m)):
                    for j in range(len(m[i])):
                        if  m[i][j] != 0:
                            temp.add((i-orig[0],j-orig[0]))

                out.append(temp)
    return out

def get_neighbourhood(s):
    out = s.copy()
    def near(x,y):
        return set(((x,y+1),(x+1,y),(x,y-1),(x-1,y)))
    for i in s:
        out = out | near(*i)
    return out - s 


T1 = [[1,0,0],
      [1,1,0],
      [0,0,0],]

T2 = [[1,0],
      [1,1],]

tiles = [T2,]


tiles = [j for i in tiles for j in generate_rotrefl(i)]
tiles_sets =[ (j, get_neighbourhood(j))  for i in tiles for j in get_coord_from_m_all_displacements(i) ]

    
def draw_from_tset_2(p, tliesss):
    s = []
    for i in range(len(p)):
        t = tliesss[p[i][0]][0]
        for a,b in t:
            s.append((a + p[i][1],b + p[i][2], i+1))
    minx, maxx = min(s, key = lambda t:t[0])[0], max(s, key = lambda t:t[0])[0]
    miny, maxy = min(s, key = lambda t:t[1])[1], max(s, key = lambda t:t[1])[1]
    t = np.zeros((maxx-minx + 1 , maxy-miny + 1), dtype = int)
    for i in s: t[i[0]-minx][i[1]-miny] = i[2]
    return t.tolist()

def tile_rect(n,m, tiles):

    grid = set([(i,j) for i in range(n) for j in range(m)])
    startborder = set()

    for i in range(n):
        startborder.add((i, 0))
        startborder.add((i, m-1))

    for i in range(m):
        startborder.add((0,i))
        startborder.add((n-1, i))

    global out
    global end
    global memo

    end = []
    out = []
    memo = set()


    def dfs(grid, border, pattern, neighbourhood, p, n):
       

        if len(out) > 0:
            return

        if len(grid) > 0 and not connected_tile(grid):
            return

        if len(grid) == 0:
            out.append(p)
            end.append(1)
            return

        if pattern != set():
            h = extract_m_from_set_directly(pattern)
            sh = int(''.join(''.join(map(str, i)) for i in h) )
            if sh in memo:
                return
            else:
                memo.add(sh)

        # if n > 5:
        #     h = extract_m_from_set_directly(pattern)
        #     print('----------------------')
        #     print(n)
        #     print_m(h)
        #     print('\n neighbourhood ')
        #     print_m(extract_m_from_set_directly(neighbourhood & border))

        target_cells = border - pattern

        if len(target_cells) > 0:

            count_r = 0

            for x,y in (target_cells ): #& neighbourhood):
                for r in range(len(tiles)):

                    t = tiles[r]
                    tempt = set((x+t1,y+t2) for t1, t2 in t[0])

                    # if n > 5:
                    #     print(f'_1 {n}, count: {count_r}' )
                    #     count_r +=1

                    if set(tempt) <= set(grid):
                        temp_p = deepcopy(p)
                        temp_p.append((r, x, y))
                        temp_pattern = pattern |  tempt
                        tempgrid = grid - tempt
               
                        tempnear = set((x+t1,y+t2) for t1, t2 in t[1])
                        temp_neneighbourhood = ((neighbourhood | tempnear) - temp_pattern) & tempgrid

                        dfs(tempgrid, border, temp_pattern, temp_neneighbourhood, temp_p, n+1)
        else:
            print('new border ', n)

            if len(grid) == 0:
                out.append(p)
                return

            tempborder = get_coord_from_m_same(get_border(extract_m_from_set_directly222(grid)))

            target_cells = tempborder - pattern

            if len(target_cells) > 0:
                for x,y in target_cells: #& neighbourhood):
                    for r in range(len(tiles)):
                        t = tiles[r]
                        tempt = set((x+t1,y+t2) for t1, t2 in t[0])
                        if set(tempt) <= set(grid):
                            temp_p = deepcopy(p)
                            temp_p.append((r, x, y))
                            temp_pattern = pattern |  tempt
                            tempgrid = grid - tempt
               
                            tempnear = set((x+t1,y+t2) for t1, t2 in t[1])
                            temp_neneighbourhood = ((neighbourhood | tempnear) - temp_pattern) & tempgrid
               
                            dfs(tempgrid, tempborder, temp_pattern, temp_neneighbourhood, temp_p, n+1)



    dfs(grid, startborder, set(), set(((0,0),)),  [],   0) 

    return out


 
solution = tile_rect(6, 6, tiles_sets)
print(solution)

if solution != [] and solution[0] != [] :
    
    ms = draw_from_tset_2(solution[0], tiles_sets)
    
    print('[', ',\n  '.join(map(str, ms)) , '],',)
else:
    print('\n\n NO SOLUTIONS')


new border  10
[[(4, 5, 4), (3, 0, 0), (7, 0, 1), (8, 3, 5), (8, 5, 5), (1, 1, 5), (3, 2, 0), (4, 5, 1), (10, 0, 3), (8, 5, 2), (8, 3, 2), (4, 3, 4)]]
[ [2, 3, 3, 9, 9, 6],
  [2, 2, 3, 9, 6, 6],
  [7, 11, 11, 12, 4, 4],
  [7, 7, 11, 12, 12, 4],
  [8, 10, 10, 1, 5, 5],
  [8, 8, 10, 1, 1, 5] ],


In [ ]:
#